# EdiPhi Task

This notebook processes an Excel file containing construction estimates. We will - 
- collect legend data
- clean the data
- convert it into a JSON structure that is ready for use

JSON structure is predefined as -

{
    "line_items": [
        {
            "sort1": "General Requirements",
            "sort2": "General Trade Requirements",
            "description": "Item 1",
            "quantity": 1,
            "uom": "each",
            "labor": 50,
            "material": 100,
            "unit_cost": 150,
            "total": 150
        },
    ]
}

## Assumptions
Following are my assumptions while solving this assignment -

- _sort1_ and _sort2_ keys in the final json will come from yellow and blue colored rows; these rows will be used to create lineItems for final JSON and work as an identifier
- labor column does not contain any data, but is still present in final output to maintain structure
- All the rows with no _total_ column value are discarded
- Legend for red colored rows says 'ignore', but the data is considered for the sake of this exercise. As by discarding the red rows, the total cost does not add up for each sort1 and sort2 key. NOTE - there is a condition put in place, in _data cleaning_ section, for this situation. Please uncomment the condition to ignore red colored rows

## Step 0: Install Necessary Libraries

To ensure that the script runs smoothly, we will begin by installing the required Python libraries.

In [7]:
### Install necessary libraries
!pip install openpyxl pandas

## Collect legends for each row

Step 1: 

Import Necessary Libraries
We will begin by importing the necessary libraries for handling Excel files and manipulating the data.
_openpyxl_ - this library allows to read cell color; used for extracting legend for each row.

- The legend information helps in identifying sort1, and sort2 keys in the final JSON format

In [8]:
## import libraries
import openpyxl
from openpyxl.styles import PatternFill

In [9]:
# Load the workbook and select the active worksheet
wb = openpyxl.load_workbook('sample_estimate.xlsx')
ws = wb.active

# Let's define the colors and corresponding text
color_map = {
    "FFFFFF": "lineItem",  # White
    "FFFFCC": "sort1",     # Light Yellow
    "CCFFFF": "sort2",     # Light Cyan
    "FFCCCC": "ignore"     # Light Pink
}

# Initialize an empty list to store legend values
legend = []

# Iterate through the rows, checking the fill color of each cell in column A
# Start from row 11 (min_row=11) since the first 10 rows are instructions
for row in ws.iter_rows(min_row=11, max_col=1):  
    cell = row[0]  # Column to get color and put text in
    
    fill = cell.fill
    if fill and fill.fgColor and fill.fgColor.type == "rgb":
        color_code = fill.fgColor.rgb[-6:]  # Extracting last 6 characters for hex color
        if color_code in color_map:
            legend.append(color_map[color_code])  # append legend information
        else:
            legend.append('lineItem')
    else:
        legend.append('lineItem')

## close the workbook after processing
wb.close()

## Build JSON structure
Now that we have legend to identify sort1 and sort2 keys correctly, Let's build the required JSON file

### Data Cleaning
We will now clean the data by loading it into a pandas DataFrame and initializing variables to track sort1, sort2, and the list of line items.

In [10]:
## import libraries
import json
import pandas as pd

In [11]:
# Load the Excel file
excel_file_path = 'sample_estimate.xlsx'
## skip first 9 rows; 10th row will give headers; data starts from row 11th
## use columns that are necessary; B to H
df = pd.read_excel(excel_file_path, skiprows=9, usecols="B:H")

We will now iterate through each row in the DataFrame, skipping rows with a total of zero, and build a JSON structure based on the sort1, sort2, and line item details.

In [12]:
# Initialize variables to track sort1 and sort2
sort1 = ''
sort2 = ''
line_items = []

for index, row in df.iterrows():
#     print(index, legend[index], row['description'])
    
    first_column_value = legend[index]
    description = row['description']
    quantity = row['quantity']
    uom = row['uom']
    labor = row['labor']
    material = row['material']
    unit_cost = row['unit_cost']
    total = row['total']
    
    if total == 0:
        continue  # Skip rows where total is zero
    if first_column_value == 'sort1':
        sort1 = description
    elif first_column_value == 'sort2':
        sort2 = description
## uncomment this condition to ignore red marked rows in the workbook
#     elif first_column_value == 'ignore':
#         continue  # Skip this row
    else:
        # If it's a line item, add the row to the line_items list
        line_items.append({
            "sort1": sort1,
            "sort2": sort2,
            "description": description,
            "quantity": quantity,
            "uom": uom,
            "labor": labor,
            "material": material,
            "unit_cost": unit_cost,
            "total": total
        })

### Output
Finally, we construct the JSON structure containing all the processed line items and save it to a file.

In [13]:
# Construct the final JSON structure
final_output = {
    "line_items": line_items
}

# Output the final result
output_json_path = 'output_result.json'
with open(output_json_path, 'w') as json_file:
    json.dump(final_output, json_file, indent=4)

print(f"Output saved to {output_json_path}")

Output saved to output_result.json


### Problems Encountered

Following were the challenges for this assginment -

#### get sort1, sort2 values:
I had to make a choice between two methods -
- Identify colors of each row and define sort1 and sort2 according to the legend. (Chosen)
- Identify pattern for knowing if a row is sort1/sort2 row and not a lineItem. I created a logic for this as - if a row would have no value (merged cell) for quantity, uom, labor, material, and unit_cost, this cell is sort1/sort2. If there are 2 continous such rows, it is sort1 and sort2, otherwise it is sort2.

I chose the first method as I believe it could scale well and is less prone to data errors.

#### reading excel file:
I tried parsing the excel file through openpyxl, but encountered an issue -
- It parses formulas as is, and did not read the value. For eg - for quantity, it said "=3*11" instead of 33
- While reading in value_only mode, it would make such values NaN
- Solved it by not modifying the excel file and reading through pandas
- Found other possible solutions but did not implement for the sake of time


## Alternatives

While attempting this problem, I realized and explored a few more solutions - 
- Create another column for legend using VBA function in excel; two functions - for identifying color of cell, for mapping each row to the provided legend
- Write add-on custom script for identifying color of cell in google sheets


## Conclusion
This exercise demonstrates -
- process of reading and processing an Excel file
- applying logic to determine the appropriate categories based on cell colors
- converting the data into a structured JSON format

The final output can be used for further analysis or integration into other systems.